STEP 1: Set up the Processing Environment. Click in the cell below, then hold down the SHIFT key, then press ENTER. This will execute the cell and install the packages needed to run this workflow.

In [ ]:
# INITIALIZE Packages (JUST DO THIS ONCE!)
pip install pandas
pip install numpy
pip install pandarallel
pip install multiprocessing
pip install matplotlib
pip install seaborn
pip install docplex
pip install cplex
pip install squarify

Step 2: Predict Mineralogy with dynamic library from VNIR-SWIR 

Click in the cell below, then hold down the SHIFT key, then press ENTER. This will execute the cell and interactive widgets will appear below this code block.

NOTE: minerals can be added to the minerla library here (see below) with elements in wt.%

In [3]:
#Geochemistry-VNIR-SWIR Integrated Predicted Mineralogy by Linear Programming
#Written by Cassady Harraden, MDRU, University of British Columbia - 18 December 2025 RELEASE

#Submitted as an appendix in paper "Dynamic Mineral Prediction from Integrated Geochemistry, pXRF, and VNIR-SWIR Spectroscopy at Drillhole and Deposit Scales"
#PAPER AUTHORS: Cassady L. Harraden, Brian A. McNulty, Shaun L.L. Barker, Carolina Marín, Sasha Wilson, Maxwell L. Porter


#To execute the code, click in this box, hold down the "SHIFT" key, and hit the "ENTER" key ---> this will pop up a series of widgets where you can upload your data, map fields, set your static mineral library, and run the predicted mineralogy
#NOTE: this code is set up to use MOST of the processing capacity of your computer - if you are doing other tasks, this will slow your computer/processing down significantly
#NOTE: the code WILL NOT warn you if an element is missing for a mineral - it will continue calulating with the other elements - so, for example, if you have zircon active, but Zr is not included in the incoming elemental data, 
#      the algroithm will only use Si and, in this case, you will have zircon mapped where quartz should probably be - YOU MUST THINK ABOUT YOUR SELECTIONS AND PROPER GEOLOGY/MINERALOGY CONTEXT FOR THIS TO WORK


# ==== Set Up User Interface ====
import io, os, sys
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from joblib import Parallel, delayed
import math


PANDARALLEL_OK = True
try:
    import multiprocessing
    from pandarallel import pandarallel
    num_processors_available = multiprocessing.cpu_count()
    num_processors_to_use = max(1, num_processors_available - 1)
    pandarallel.initialize(nb_workers=num_processors_to_use, progress_bar=True)
except Exception as e:
    PANDARALLEL_OK = False
    print("WARNING: pandarallel not available; will run single-core. May take more time.", e)

SQUARIFY_OK = True
try:
    import squarify
except Exception as e:
    SQUARIFY_OK = False
    print("WARNING: squarify not available; treemap will be skipped.", e)

import matplotlib.pyplot as plt
import seaborn as sns

#Mineral library (CAN BE EDITED BY USER FOLLOWING CONVENTIONS BELOW)
mineral_elements = {
    "Albite": {"Si_pct": 31.5, "Al_pct": 10.77, "Na_pct": 8.3, "Ca_pct": 0.76},
    "Analcime": {"Si_pct": 25.51, "Na_pct": 10.44, "Al_pct": 12.26},
    "Anhydrite": {"S_pct": 23.55, "Ca_pct": 29.44},
    "Biotite": {"Si_pct": 19.44, "Mg_pct": 14.02, "K_pct": 9.02, "Fe_pct": 6.44, "Al_pct": 6.22},
    "Calcite": {"Ca_pct": 40.04, "C_pct": 12.0},
    "Chalcopyrite": {"S_pct": 34.94, "Fe_pct": 30.43, "Cu_pct": 34.64},
    "Clinochlore": {"Si_pct": 14.16, "Al_pct": 9.07, "Mg_pct": 15.31, "Fe_pct": 11.73},
    "Diaspore": {"Al_pct": 44.98},
    "Dickite": {"Al_pct": 20.90, "Si_pct": 21.76},
    "Dolomite": {"Ca_pct": 21.73, "Mg_pct": 13.18, "C_pct": 13.33},
    "Epidote": {"Si_pct": 16.22, "Fe_pct": 24.20, "Ca_pct": 15.44, "Al_pct": 3.90},
    "Gypsum": {"Ca_pct": 23.28, "S_pct": 18.62},
    "Illite": {"K_pct": 6.03, "Mg_pct": 1.87, "Al_pct": 9.01, "Fe_pct": 1.43, "Si_pct": 25.25},
    "Kaolinite": {"Si_pct": 21.76, "Al_pct": 20.90},
    "Orthoclase": {"Si_pct": 30.27, "Al_pct": 9.69, "K_pct": 14.05},
    "Labradorite": {"Si_pct": 24.8, "Na_pct": 3.38, "Al_pct": 15.88, "Ca_pct": 8.85},
    "Magnesiohornblende": {"Si_pct": 23.94, "Mg_pct": 11.84, "Fe_pct": 1.7, "Ca_pct": 9.67, "Al_pct": 5.75},
    "Magnetite": {"Fe_pct": 72.36},
    "Molybdenite": {"S_pct": 40.06, "Mo_pct": 59.94},
    "Muscovite": {"Si_pct": 21.13, "K_pct": 9.81, "Al_pct": 20.30},
    "Paragonite":{"Si_pct": 22.05, "Al_pct": 21.18, "Na_pct": 6.02}, 
    "Phengite":{"Si_pct": 27.76, "Al_pct": 6.67, "K_pct": 9.66, "Mg_pct": 4.51, "Fe_pct": 3.45},
    "Plagioclase": {"Si_pct": 31.12, "Na_pct": 4.25, "Ca_pct": 7.4, "Al_pct": 9.96},
    "Pyrite": {"S_pct": 53.45, "Fe_pct": 46.55},
    "Pyrophyllite": {"Si_pct": 31.18, "Al_pct": 14.98},
    "Quartz": {"Si_pct": 46.74},
    "Sanidine": {"Si_pct": 30.72, "K_pct": 10.69, "Na_pct": 2.1, "Al_pct": 9.84},    
    "Tennantite": {"Fe_pct": 3.8, "Cu_pct": 47.51, "S_pct": 28.33, "As_pct": 20.37},
    "Apatite": {"P_pct": 18.25, "Ca_pct": 39.36},
    "Rutile": {"Ti_pct": 59.94},
    "Titanite": {"Si_pct": 14.2, "Ca_pct": 19.25, "Ti_pct": 18.16},
    "Mg-chlorite": {"Si_pct": 12.62, "Ti_pct": 0.12, "Al_pct": 11.59, "Fe_pct": 11.33, "Ca_pct": 0.07, "Mn_pct": 1.01, "Mg_pct": 12.90},
    "Fe-chlorite": {"Si_pct": 10.75, "Ti_pct": 0.12, "Al_pct": 10.27, "Mn_pct": 1.01, "Ca_pct": 0.07, "Mg_pct": 2.23, "Fe_pct": 32.8},
    "Montmorillonite": {"Na_pct": 0.84, "Ca_pct": 0.73, "Al_pct": 9.83, "Si_pct": 20.46}, 
    "Siderite": {"Fe_pct": 48.20, "C_pct": 10.37},
    "Ankerite": {"Ca_pct": 19.42, "Mg_pct": 3.53, "Mn_pct": 2.66, "Fe_pct": 16.24, "C_pct": 11.64},
    "Magnesite": {"Mg_pct": 28.83, "C_pct": 14.25}
}

default_active_minerals = ["Apatite", "Rutile", "Titanite", "Quartz", "Pyrite", "Albite",
                           "Chalcopyrite", "Molybdenite", "Epidote", "Gypsum", "Tennantite",
                           "Magnesiohornblende", "Sanidine", "Orthoclase", "Plagioclase", "Magnetite"] 

expected_element_keys = sorted({ek for m in mineral_elements.values() for ek in m.keys()})

#Oxide to Element conversion
atomic = {'Si': 28.085, 'Al': 26.982, 'Fe': 55.845, 'Mg': 24.305, 'Ca': 40.078, 'Na': 22.990, 'K': 39.098, 'Ti': 47.867, 'P': 30.974, 'Mn': 54.938, 'O': 15.999, 'C': 12.011}
oxide_factors = {
    'SiO2': atomic['Si']/(atomic['Si'] + 2*atomic['O']),
    'Al2O3': (2*atomic['Al'])/(2*atomic['Al'] + 3*atomic['O']),
    'FeO': atomic['Fe']/(atomic['Fe'] + atomic['O']),
    'Fe2O3': (2*atomic['Fe'])/(2*atomic['Fe'] + 3*atomic['O']),
    'MgO': atomic['Mg']/(atomic['Mg'] + atomic['O']),
    'CaO': atomic['Ca']/(atomic['Ca'] + atomic['O']),
    'Na2O': (2*atomic['Na'])/(2*atomic['Na'] + atomic['O']),
    'K2O': (2*atomic['K'])/(2*atomic['K'] + atomic['O']),
    'TiO2': atomic['Ti']/(atomic['Ti'] + 2*atomic['O']),
    'P2O5': (2*atomic['P'])/(2*atomic['P'] + 5*atomic['O'])
}

#Set Up interactive widgets
title = widgets.HTML("<h2>Dynamic Mineral Predictions</h2>")
note = widgets.HTML("<em>Mapping dropdowns populate after you upload a file.</em>")
file_upload = widgets.FileUpload(accept='.csv,.xlsx', multiple=False, description='Upload data')

oxides_checkbox = widgets.Checkbox(value=False, description='Inputs are oxides (wt%)')
ppm_to_pct_checkbox = widgets.Checkbox(value=False, description='Convert ppm→% (÷1e4)')
residual_slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Residuals% threshold')
exclude_100_residuals = widgets.Checkbox(value=False, description='Exclude 100% residual rows')

swir_a_col = widgets.Dropdown(options=['(select after upload)'], description='SWIR A')
swir_b_col = widgets.Dropdown(options=['(select after upload)'], description='SWIR B')
swir_c_col = widgets.Dropdown(options=['(select after upload)'], description='SWIR C')

active_minerals_select = widgets.SelectMultiple(
    options=sorted(mineral_elements.keys()),
    value=tuple(default_active_minerals),
    description='Active minerals',
    rows=10
)

rule_col = widgets.Dropdown(options=['(select after upload)'], description='Rule column')
rule_op = widgets.Dropdown(options=['equals', 'in'], description='Operator')
rule_vals = widgets.Text(value='', description='Values (comma‑sep)')
rule_remove = widgets.SelectMultiple(options=sorted(mineral_elements.keys()), description='Remove minerals', rows=8)
add_rule_btn = widgets.Button(description='Add rule', button_style='info')
rules_output = widgets.Output()

mapping_widgets = {}
mapping_box = widgets.VBox([widgets.HTML("<h4>Map your columns to expected fields</h4>"),
                            widgets.HTML("<em>(appears after upload)</em>")])

run_btn = widgets.Button(description='Run Mineral Prediction (LP) & Export', button_style='success')
export_name = widgets.Text(value='LP_output.csv', description='Export CSV')
status_out = widgets.Output()
plots_out = widgets.Output()
preview_out = widgets.Output()

#Collect global parameters for workers
CURRENT_ACTIVE_MINERALS = []
CURRENT_RULES = []
rules = []

def reset_ui_after_upload(df_cols):
    columns = ['(none)'] + list(df_cols)
    kids = [widgets.HTML("<h4>Map your columns to expected fields</h4>")]
    for key in expected_element_keys:
        default_choice = key if key in df_cols else '(none)'
        dd = widgets.Dropdown(options=columns, value=default_choice, description=key)
        mapping_widgets[key] = dd
        kids.append(dd)
    mapping_box.children = kids
    swir_a_col.options = ['(none)'] + list(df_cols)
    swir_b_col.options = ['(none)'] + list(df_cols)
    swir_c_col.options = ['(none)'] + list(df_cols)
    rule_col.options = ['(none)'] + list(df_cols)

def parse_uploaded_df():
    if not file_upload.value:
        return None
    blob = next(iter(file_upload.value.values()))
    name = blob['metadata']['name']
    content = blob['content']
    try:
        if name.lower().endswith('.csv'):
            df = pd.read_csv(io.BytesIO(content))
        else:
            df = pd.read_excel(io.BytesIO(content), engine='openpyxl')
        return df
    except Exception as e:
        with status_out:
            print(f"WARNING: Failed to read file: {e}")
        return None

def detect_ppm_cols(df):
    return [c for c in df.columns if ('ppm' in c.lower()) or c.lower().endswith('_ppm')]

def apply_units_conversion(df, use_oxides=False):
    df = df.copy()
    if use_oxides:
        oxide_to_elem_key = {
            'SiO2': 'Si_pct', 'Al2O3': 'Al_pct', 'FeO': 'Fe_pct', 'Fe2O3': 'Fe_pct',
            'MgO': 'Mg_pct', 'CaO': 'Ca_pct', 'Na2O': 'Na_pct', 'K2O': 'K_pct',
            'TiO2': 'Ti_pct', 'P2O5': 'P_pct'
        }
        for oxide, factor in oxide_factors.items():
            if oxide in df.columns:
                target_key = oxide_to_elem_key.get(oxide)
                if target_key and target_key not in df.columns:
                    df[target_key] = df[oxide].fillna(0) * factor
    if ppm_to_pct_checkbox.value:
        for c in detect_ppm_cols(df):
            newc = c.replace('ppm', '').strip() + '_pct'
            if newc not in df.columns:
                df[newc] = df[c] / 1e4
    return df

#VNIR-SWIR handling
swir_synonyms = {
    'illite': 'Illite', 'kaolinite': 'Kaolinite',
    'k-spar': 'K-feldspar', 'k-feldspar': 'K-feldspar', 'kspar': 'K-feldspar',
    'muscovite': 'Muscovite', 'phengite': 'Phengite', 'paragonite': 'Paragonite',
    'albite': 'Albite', 'plagioclase': 'Plagioclase',
    'calcite': 'Calcite', 'pyrophyllite': 'Pyrophyllite',
    'montmorillonite': 'Montmorillonite', 'magnesite': 'Magnesite',
    'mg-chlorite': 'Mg-chlorite', 'fe-chlorite': 'Fe-chlorite'
}
def normalize_swir_name(name):
    if pd.isna(name): return None
    s = str(name).strip()
    if s == '' or s == '0': return None
    if s in mineral_elements: return s
    key = swir_synonyms.get(s.lower())
    if key and key in mineral_elements: return key
    for lib_name in mineral_elements.keys():
        if lib_name.lower() == s.lower():
            return lib_name
    return None

def build_mapping(df):
    df_mapped = df.copy()
    for key, widget in mapping_widgets.items():
        src = widget.value
        if src != '(none)':
            if src != key:
                df_mapped[key] = df_mapped[src]
        else:
            # leave NaN to mean “not measured”
            if key not in df_mapped.columns:
                df_mapped[key] = np.nan
    # SWIR mapping (text; keep NaN if unmapped)
    for label, w in [('SWIR A', swir_a_col), ('SWIR B', swir_b_col), ('SWIR C', swir_c_col)]:
        if w.value and w.value != '(none)' and w.value in df_mapped.columns:
            df_mapped[label] = df_mapped[w.value]
        else:
            df_mapped[label] = np.nan
    return df_mapped

def apply_geologic_rules(row, active_list):
    updated = set(active_list)
    for r in CURRENT_RULES:  #calls collected global parameters
        col, op, vals, to_remove = r['col'], r['op'], r['vals'], r['remove']
        if col in row.index:
            val = row[col]
            if op == 'equals' and val in vals:
                updated -= set(to_remove)
            elif op == 'in' and val in vals:
                updated -= set(to_remove)
    return list(updated)

def process_row_wrapped(row):
    # Import docplex inside the worker (parallel-safe)
    try:
        from docplex.mp.model import Model
    except Exception:
        return {'Residuals%': 100.0, 'HighestProp': None}

    #Build row-specific active list from base (snapshot) + VNIR-SWIR additions + geological rules ---
    row_minerals = []
    for mineral_column in ["SWIR A", "SWIR B", "SWIR C"]:
        nm = normalize_swir_name(row.get(mineral_column, np.nan))
        if nm is not None:
            row_minerals.append(nm)

    base_active = list(CURRENT_ACTIVE_MINERALS)
    active_minerals = base_active + list(set(row_minerals) - set(base_active))
    active_minerals = apply_geologic_rules(row, active_minerals)

    #Predicted mineralogy by LP Optimization model
    m = Model(name='Mineralogy')
    minerals = {mineral: m.continuous_var(name=mineral, lb=0) for mineral in active_minerals}

    elements = {}
    for mineral in active_minerals:
        for element, _ in mineral_elements[mineral].items():
            if element in row.index and pd.notnull(row[element]):
                elements[element] = row[element]

    # Handle missing or zero Si --> no mineralogy calculated
    if 'Si_pct' in row and (pd.isna(row['Si_pct']) or row['Si_pct'] == 0):
        sol_dict = {f"{mineral}%": 0.0 for mineral in active_minerals}
        for element in elements:
            sol_dict[f"Residual_{element}"] = elements[element]
        sol_dict['Residuals%'] = 100.0
        sol_dict['HighestProp'] = None
        return sol_dict

    for element in elements:
        element_constraint = m.sum(
            minerals[mineral] * mineral_elements[mineral].get(element, 0.0)
            for mineral in active_minerals if element in mineral_elements[mineral]
        )
        m.add_constraint(element_constraint <= elements[element], f"{element}_constraint")

    m.add_constraint(m.sum(minerals.values()) <= 1.0, "TotalMineralConstraint")
    m.maximize(m.sum(minerals.values()))

    sol_dict = {}
    if m.solve():
        for mineral in active_minerals:
            sol_dict[f"{mineral}%"] = minerals[mineral].solution_value * 100.0
        sol_dict['Residuals%'] = 100.0 - sum(v for k, v in sol_dict.items() if k.endswith('%') and k != 'Residuals%')
        # HighestProp: pick the mineral with max %, exclude Residuals%
        candidates = [k for k in sol_dict if k.endswith('%') and k != 'Residuals%']
        sol_dict['HighestProp'] = max(candidates, key=lambda k: sol_dict[k], default=None)

        # Optional: per-element residuals
        for element in elements:
            accounted = sum(
                minerals[mineral].solution_value * mineral_elements[mineral].get(element, 0.0)
                for mineral in active_minerals if element in mineral_elements[mineral]
            )
            residual = elements[element] - accounted
            if residual < 0 and abs(residual) < 1e-6:
                residual = 0.0
            sol_dict[f"Residual_{element}"] = residual
    else:
        sol_dict = {f"{mineral}%": 0.0 for mineral in active_minerals}
        for element in elements:
            sol_dict[f"Residual_{element}"] = elements[element]
        sol_dict['Residuals%'] = 100.0
        sol_dict['HighestProp'] = None

    return sol_dict

def on_add_rule_clicked(_):
    with rules_output:
        rules_output.clear_output()
        vals = [v.strip() for v in rule_vals.value.split(',') if v.strip()]
        if rule_col.value == '(none)' or len(vals) == 0 or len(rule_remove.value) == 0:
            print("WARNING: Select rule column, enter values, and choose minerals to remove.")
            return
        rule = {'col': rule_col.value, 'op': rule_op.value, 'vals': vals, 'remove': list(rule_remove.value)}
        rules.append(rule)
        print(f"Added rule: if {rule_col.value} {rule_op.value} {vals} -> remove {list(rule_remove.value)}")
add_rule_btn.on_click(on_add_rule_clicked)

def on_upload_change(_):
    clear_output(wait=True)
    display(ui)
    df = parse_uploaded_df()
    if df is not None:
        # Convert 'null' strings to NaN immediately for preview and mapping
        df = df.replace({'null': np.nan, 'NULL': np.nan, 'Null': np.nan})
        reset_ui_after_upload(df.columns)
        with preview_out:
            preview_out.clear_output()
            print("SUCCESS! File loaded. Preview:")
            display(df.head())
file_upload.observe(on_upload_change, names='value')

def run_pipeline(_):
    with status_out:
        status_out.clear_output()
        df = parse_uploaded_df()
        if df is None:
            print("WARNING: Upload a data file (.csv or .xlsx) first."); return
        # Convert string nulls to NaN
        df = df.replace({'null': np.nan, 'NULL': np.nan, 'Null': np.nan})

        # Oxides hint
        known_oxides = set(oxide_factors.keys())
        present_oxides = known_oxides.intersection(set(df.columns))
        if oxides_checkbox.value and len(present_oxides) == 0:
            print("WARNING: Oxides mode selected, but no known oxide columns were found (SiO2, Fe2O3, ...).")
        if not oxides_checkbox.value and len(present_oxides) > 0:
            print(f"WARNING: Detected oxide-like columns {sorted(present_oxides)}. "
                  f"If these are oxides in wt%, tick the checkbox.")

        # Units conversion precedes mapping
        try:
            df_units = apply_units_conversion(df, use_oxides=oxides_checkbox.value)
        except Exception as e:
            print(f"WARNING: Units conversion failed: {e}"); return

        # Schema mapping
        try:
            df_mapped = build_mapping(df_units)
        except Exception as e:
            print(f"WARNING: Column mapping failed: {e}"); return

        # Keep elemental & SWIR columns as NaN; zero-fill only non-element/non-SWIR if desired
        swir_cols = ['SWIR A', 'SWIR B', 'SWIR C']
        element_cols = set(expected_element_keys)
        non_sensitive_cols = [c for c in df_mapped.columns if c not in swir_cols and c not in element_cols]
        df_mapped[non_sensitive_cols] = df_mapped[non_sensitive_cols].fillna(0)

        # Snapshot state for workers
        global CURRENT_ACTIVE_MINERALS, CURRENT_RULES
        CURRENT_ACTIVE_MINERALS = list(active_minerals_select.value)
        CURRENT_RULES = list(rules)

        # LP execution (parallel if available)
        try:
            if PANDARALLEL_OK:
                result_series = df_mapped.parallel_apply(process_row_wrapped, axis=1)
            else:
                result_series = df_mapped.apply(process_row_wrapped, axis=1)
        except Exception as e:
            # print(f"WARNING: LP execution failed ({e}); retrying single-core. Predicted mineralogy will still run, but might be slow.")
            result_series = df_mapped.apply(process_row_wrapped, axis=1)

        result_df = pd.DataFrame(list(result_series))
        output_df = pd.concat([df_mapped.reset_index(drop=True), result_df], axis=1)


        # Column ordering
        percentage_columns = sorted([c for c in output_df.columns if '%' in c and not c.startswith("Residual_")])
        residual_columns = sorted([c for c in output_df.columns if c.startswith("Residual_")])
        non_percentage_columns = [c for c in output_df.columns if '%' not in c and not c.startswith("Residual_") and c != 'HighestProp']
        final_columns_order = non_percentage_columns + percentage_columns + residual_columns
        if 'HighestProp' in output_df.columns:
            final_columns_order.append('HighestProp')
        output_df = output_df[final_columns_order]

        # Export
        export_path = export_name.value
        if not os.path.isabs(export_path): export_path = os.path.join(os.getcwd(), export_path)
        try:
            output_df.to_csv(export_path, index=False)
            print(f"SUCCESS! Exported: {export_path}")
        except Exception as e:
            print(f"WARNING: Failed to export CSV: {e}")
            return

    # Plots
    plots_out.clear_output()
    with plots_out:
        try:
            filt = output_df['Residuals%'] > residual_slider.value
            if exclude_100_residuals.value:
                filt &= (output_df['Residuals%'] < 100)
            high_residuals_df = output_df[filt]

            elemental_columns = [c for c in output_df.columns if c.endswith('pct') and not c.startswith('Residual_')]
            elemental_data_for_plotting = high_residuals_df[elemental_columns]
            num_rows = len(elemental_data_for_plotting)

            plt.figure(figsize=(18, 8))
            sns.boxplot(data=elemental_data_for_plotting)
            plt.xticks(rotation=45)
            plt.title(f'Box & Whisker: Elemental components for Residuals ≥ {residual_slider.value}%')
            plt.ylabel('Weight %'); plt.xlabel('Element')
            plt.annotate(f'n={num_rows}', xy=(1, 0), xycoords='axes fraction',
                         ha='right', va='bottom', fontsize=12, xytext=(-10, 10), textcoords='offset points')
            plt.tight_layout(); plt.show()

            mineral_percentage_columns = [c for c in result_df.columns if '%' in c and c != 'Residuals%']
            total_minerals_percent = result_df[mineral_percentage_columns].sum()
            total_minerals_percent_normalized = (total_minerals_percent / total_minerals_percent.sum()) * 100
            total_minerals_percent_normalized_sorted = total_minerals_percent_normalized.sort_index()

            plt.figure(figsize=(18, 8))
            total_minerals_percent_normalized_sorted.plot(kind='bar')
            plt.title('Modal Mineralogy (normalized across dataset)')
            plt.ylabel('Percentage (%)'); plt.xlabel('Mineral')
            plt.xticks(rotation=45)
            for idx, val in enumerate(total_minerals_percent_normalized_sorted):
                plt.text(idx, val, f'{val:.2f}%', ha='center', va='bottom')
            plt.tight_layout(); plt.show()

            if SQUARIFY_OK:
                filtered = total_minerals_percent_normalized[total_minerals_percent_normalized > 0]
                values = filtered.values; labels = filtered.index
                colors = [plt.cm.tab20(i/float(len(labels))) for i in range(len(labels))]
                plt.figure(figsize=(8, 8))
                squarify.plot(sizes=values, label=labels, color=colors, alpha=0.6, text_kwargs={'fontsize':8})
                plt.title('Modal Mineralogy Treemap')
                plt.annotate(f'n={result_df.shape[0]}', xy=(0.95, 0.05), xycoords='axes fraction',
                             ha='right', va='bottom', fontsize=12)
                plt.axis('off'); plt.tight_layout(); plt.show()
            else:
                print("WARNING: Treemap skipped (squarify not installed).")
        except Exception as e:
            print(f"WARNING: Plotting failed: {e}")

    preview_out.clear_output()
    with preview_out:
        print("Output preview:")
        display(output_df.head())

add_rule_btn.on_click(on_add_rule_clicked)
file_upload.observe(on_upload_change, names='value')
run_btn.on_click(run_pipeline)

# Assemble UI
ui = widgets.VBox([
    title,
    note,
    widgets.HTML("<b>Step 1:</b> Upload your data (.csv or .xlsx)"),
    file_upload,
    widgets.HTML("<b>Step 2:</b> Set units & conversions (BE SURE TO CHECK CONVERSION OUTPUTS AFTER PROCESSING"),
    widgets.HBox([oxides_checkbox, ppm_to_pct_checkbox]),
    widgets.HTML("<b>Step 3:</b> Map columns to expected elemental names"),
    mapping_box,
    widgets.HTML("<b>Optional:</b> Map VNIR-SWIR columns"),
    widgets.HBox([swir_a_col, swir_b_col, swir_c_col]),
    widgets.HTML("<b>Step 4:</b> Choose static mineral library"),
    active_minerals_select,
    widgets.HTML("<b>Optional:</b> Add geologic constraint rules"),
    widgets.HBox([rule_col, rule_op, rule_vals]),
    widgets.HBox([rule_remove, add_rule_btn]),
    rules_output,
    widgets.HTML("<b>Step 5:</b> Plotting options"),
    widgets.HBox([residual_slider, exclude_100_residuals]),
    widgets.HTML("<b>Step 6:</b> Run LP & Export"),
    widgets.HBox([export_name, run_btn]),
    status_out,
    plots_out,
    preview_out
])

display(ui)


INFO: Pandarallel will run on 19 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
